In [1]:
import os
os.chdir('/Users/samir/Documents/Winter2023/MAST680/Assignment1')
from PIL import Image
from scipy import linalg

## Import raw video as a 4-dimensional matrix (now handles conversion to grayscale)

In [2]:
import cv2
import numpy as np

cap = cv2.VideoCapture('monte_carlo_low.mp4')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))
greybuf = np.empty((frameCount, frameHeight, frameWidth), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, buf[fc] = cap.read()
    greybuf[fc] = cv2.cvtColor(buf[fc], cv2.COLOR_BGR2GRAY) #Converts each frame to grayscale
    fc += 1

cap.release()

#cv2.namedWindow('frame 10')
#cv2.imshow('frame 10', buf[9])

#cv2.waitKey(5000)

## Dimensions: (frames, frame height, frame width, RGB)

In [3]:
print(np.shape(buf))
frames = np.shape(buf)[0]
height = np.shape(buf)[1]
width = np.shape(buf)[2]
RGB = np.shape(buf)[3]

(379, 540, 960, 3)


## Preprocessing: stacking rows

In [4]:
X = np.reshape(buf, (frames,height*width,RGB))
X_grey = np.reshape(greybuf, (frames,height*width))

## (OPTIONAL) preprocessing: convert to greyscale

In [13]:
#We want to loop through the entries of our 2D matrix 
#(thinking of the RGB component as being embedded in each entry)
#and perform a weighted average in each entry according to the following rule: 
#X_ij = 0.299R + 0.587G + 0.114B

#Create an empty 2D list of the correct dimensions (height*width, frames):
X_grey = np.zeros((frames,height*width))

for i in range(frames):
    for j in range(height*width):
        X_grey[i,j] = 0.299*X[i,j,0]+0.587*X[i,j,1]+0.114*X[i,j,2]
        
#This is more of a proof of concept. Obviously this is super inefficient code...
#I should eventually try to integrate this into the video import or use a built-in function.

In [71]:
print(X_grey.shape)

(379, 518400)


## Reformat as video frame and image print

In [ ]:
grey_image = np.reshape(X_grey,((frames, height, width)))
img = Image.fromarray(np.uint8(grey_image[10]) , 'L')
img.show()

## DMD (Warning: takes a long time to run this cell)

In [ ]:
## We want to find the DMD matrix A that satisfies the argmin problem. This involves computing
## the Moore-Penrose pseudoinverse X_dagger and multiplying on the left by the shifted matrix Y 
## whose columns are the next time step from those in the X matrix.

X = X_grey.T[:,0:378]
Y = X_grey.T[:,1:379]
A = np.matmul(Y,linalg.pinv(X))